In [1]:
import pandas as pd
import wget

In [2]:
url = "https://www.ers.usda.gov/webdocs/DataFiles/50048/FeedGrains.zip"

In [3]:
file = wget.download(url)

In [4]:
print(file)

FeedGrains.zip


In [6]:
!unzip FeedGrains.zip

Archive:  FeedGrains.zip
  inflating: FeedGrains.csv          


In [7]:
df = pd.read_csv('FeedGrains.csv', nrows=100)

In [8]:
df.head()

,SC_Group_ID,SC_Group_Desc,SC_GroupCommod_ID,SC_GroupCommod_Desc,SC_Geography_ID,SortOrder,SC_GeographyIndented_Desc,SC_Commodity_ID,SC_Commodity_Desc,SC_Attribute_ID,SC_Attribute_Desc,SC_Unit_ID,SC_Unit_Desc,Year_ID,SC_Frequency_ID,SC_Frequency_Desc,Timeperiod_ID,Timeperiod_Desc,Amount
0,2,Supply and use,9,Barley,1,0.8,United States,1,Barley,1,Planted acreage,2,Million acres,1926,3,Annual,69,Commodity Market Year,8.796
1,2,Supply and use,9,Barley,1,0.8,United States,1,Barley,1,Planted acreage,2,Million acres,1927,3,Annual,69,Commodity Market Year,9.513
2,2,Supply and use,9,Barley,1,0.8,United States,1,Barley,1,Planted acreage,2,Million acres,1928,3,Annual,69,Commodity Market Year,12.828
3,2,Supply and use,9,Barley,1,0.8,United States,1,Barley,1,Planted acreage,2,Million acres,1929,3,Annual,69,Commodity Market Year,14.703
4,2,Supply and use,9,Barley,1,0.8,United States,1,Barley,1,Planted acreage,2,Million acres,1930,3,Annual,69,Commodity Market Year,13.581
